In [1]:
import numpy as np
import pyDR as DR
from pyDR.Project import Project
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
import pickle

In [2]:
ile_resn = [3, 13, 23, 30, 36, 44, 61]
test_list = ["test_001", "test_002", "test_003", "test_004", "test_005", "test_006", "test_007","test_008", "test_009", "test_010", "test_011"]

In [3]:
def get_frame_analysis(test):
    tf=None
    #tf=1000
    molsys=DR.MolSys('./1ubq_data/1ubq_frame.pdb', './1ubq_data/'+test+'/run_001/traj_4us_PBC_fit.xtc',tf=tf)
    select=DR.MolSelect(molsys)

    frames=list()
    frames.append({'Type':'methylCC','Nuc':'ivl', 'resids':ile_resn,'sigma':0})
    frames.append({'Type':'side_chain_chi','Nuc':'ivl', 'resids':ile_resn,'n_bonds':1,'sigma':0})
    frames.append({'Type':'side_chain_chi','Nuc':'ivl', 'resids':ile_resn,'n_bonds':2,'sigma':0})

    "Frames with post-process smoothing"
    frames.append({'Type':'hops_3site','Nuc':'ivl', 'resids':ile_resn,'sigma':5})
    frames.append({'Type':'methylCC','Nuc':'ivl', 'resids':ile_resn,'sigma':5})
    frames.append({'Type':'chi_hop','Nuc':'ivl', 'resids':ile_resn,'n_bonds':1,'sigma':50})
    frames.append({'Type':'side_chain_chi','Nuc':'ivl', 'resids':ile_resn,'n_bonds':1,'sigma':50})
    frames.append({'Type':'chi_hop','Nuc':'ivl', 'resids':ile_resn,'n_bonds':2,'sigma':50})
    frames.append({'Type':'side_chain_chi','Nuc':'ivl', 'resids':ile_resn,'n_bonds':2,'sigma':50})

    select.select_bond(Nuc='ivl', resids=ile_resn)
    fr_obj=DR.Frames.FrameObj(select)
    fr_obj.tensor_frame(sel1=1,sel2=2)
    for f in frames:fr_obj.new_frame(**f)
    fr_obj.load_frames(n=-1)
    fr_obj.post_process()

    tf = molsys._traj.tf
    print(f"tf: {tf}")

    include=np.zeros([3,9],dtype=bool)
    include[0][:3]=True  
    include[1][[4,6,8]]=True
    include[2][3:]=True
    t=np.arange(tf>>1)*.005
    data=fr_obj.frames2data(mode='full')
    proj=Project('./out_figures/'+test+'.nosync',create=True)
    for d in data:proj.append_data(d)
    proj[0].detect.r_no_opt(10)
    proj.fit(bounds=False)
    proj['no_opt'][0].detect.r_auto(5)
    proj['no_opt'].fit()
    return [fr_obj.frames2ct(include=x, mode='full') for x in include]
    

In [4]:
for test in test_list:
    try:
        out = get_frame_analysis(test)
        pickle.dump(out, open("./out_figures/ile/"+test+"/out4.p", 'wb'))
    except Exception as e:
        print(e)
        pass

'Trajectory' object has no attribute '__tf'
'Trajectory' object has no attribute '__tf'
'Trajectory' object has no attribute '__tf'
'Trajectory' object has no attribute '__tf'
File does not exist: b'/home/mlehner/pyDR/tests/1ubq_data/test_005/run_001/traj_4us_PBC_fit.xtc'


Exception ignored in: <function ReaderBase.__del__ at 0x7fab7b7c7f80>
Traceback (most recent call last):
  File "/home/mlehner/.conda/envs/pyDR/lib/python3.7/site-packages/MDAnalysis/coordinates/base.py", line 2180, in __del__
    self.close()
  File "/home/mlehner/.conda/envs/pyDR/lib/python3.7/site-packages/MDAnalysis/coordinates/XDR.py", line 178, in close
    self._xdr.close()
AttributeError: 'XTCReader' object has no attribute '_xdr'


The topology and XTC trajectory files don't have the same number of atoms!
Topology number of atoms 1231
Trajectory: /home/mlehner/pyDR/tests/1ubq_data/test_006/run_001/traj_4us_PBC_fit.xtc Number of atoms 1155
'Trajectory' object has no attribute '__tf'
'Trajectory' object has no attribute '__tf'
'Trajectory' object has no attribute '__tf'
'Trajectory' object has no attribute '__tf'
'Trajectory' object has no attribute '__tf'


In [5]:
out1=pickle.load(open("./out_figures/ile/"+test_list[-1]+"/out4.p", 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: './out_figures/ile/test_011/out4.p'

In [ ]:
out1[0]["ct_prod"].shape

(42, 200000)